In [1]:
require "narray"
require "daru"
include Daru
nil

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

In [2]:
input = "../DOAS/20160205_NO2.csv"

order = ["index", "timestamp", "no2", "no2_err", "h2o", "h2o_err"]

molar_masses = {no2: 46, h2o: 18}

molar_volume = 22.4

from_ppb = true #else from ug/m^3

nil

In [3]:
d = DataFrame.from_csv(input, order: order)
d.vectors = Index.new(d.vectors.to_a.map{|v| v.to_sym})
d.head

,index,timestamp,no2,no2_err,h2o,h2o_err
0,0,2016-02-05T10:50:03+01:00,44.5446578980823,0.234613687001027,4459813.91545928,1336999.28416624
1,1,2016-02-05T10:50:06+01:00,44.0685080941092,0.216366309196476,5320194.21469098,1233330.26784266
2,2,2016-02-05T10:50:09+01:00,44.2942487501008,0.213769961742153,5954262.05497361,1218412.03898882
3,3,2016-02-05T10:50:11+01:00,44.8006275735036,0.209245577008174,5131135.20994348,1192250.84792879
4,4,2016-02-05T10:50:13+01:00,45.8245316233237,0.214061139960362,6284206.82671725,1219058.68169088
5,5,2016-02-05T10:50:16+01:00,46.771858644332,0.212441843973984,5220780.05523918,1209231.78568218
6,6,2016-02-05T10:50:18+01:00,47.3687131432783,0.200352862813292,4638697.49447677,1140073.32556966
7,7,2016-02-05T10:50:20+01:00,47.355057416822,0.213131848119867,5130443.65116201,1212798.66290045
8,8,2016-02-05T10:50:22+01:00,47.7473843478201,0.222221042429119,4946500.90407277,1264236.40292817
9,9,2016-02-05T10:50:25+01:00,47.9030874974096,0.213926222560041,6968532.09025212,1216992.37492484


In [4]:
coefficients= molar_masses.map do |m| 
  m[1] = m[1].to_f/molar_volume
  m
end

[[:no2, 2.053571428571429], [:h2o, 0.8035714285714286]]

In [5]:
if from_ppb
  coefficients.each do |chem, corr|
    d[chem] = d[chem]/corr
    err = chem.to_s.concat("_err").to_sym
    d[err] = d[err]/corr
  end
else
  coefficients.each do |chem, corr|
    d[chem] = d[chem]*corr
    err = chem.to_s.concat("_err").to_sym
    d[err] = d[err]*corr
  end
end
d.head

,index,timestamp,no2,no2_err,h2o,h2o_err
0,0,2016-02-05T10:50:03+01:00,21.691311672109638,0.11424666497441313,5549990.6503493255,1663821.3314068762
1,1,2016-02-05T10:50:06+01:00,21.45944741974013,0.10536098534784917,6620686.133837664,1534810.9999819768
2,2,2016-02-05T10:50:09+01:00,21.56937330439691,0.10409667702226579,7409748.335078269,1516246.0929638648
3,3,2016-02-05T10:50:11+01:00,21.815957774923486,0.10189349836919777,6385412.705707441,1483689.9440891608
4,4,2016-02-05T10:50:13+01:00,22.314554529618494,0.10423846815461105,7820346.273248133,1517050.803881984
5,5,2016-02-05T10:50:16+01:00,22.775861600718187,0.10344994141341828,6496970.735408757,1504821.777737824
6,6,2016-02-05T10:50:18+01:00,23.06650379150943,0.09756313319603784,5772601.326459981,1418757.9162644655
7,7,2016-02-05T10:50:20+01:00,23.05985404645245,0.10378594343228305,6384552.099223834,1509260.5582761155
8,8,2016-02-05T10:50:22+01:00,23.250900204155872,0.1082119858785275,6155645.56951278,1573271.9680883894
9,9,2016-02-05T10:50:25+01:00,23.326720868303802,0.10417276924662866,8671951.045647083,1514479.3999064674


In [6]:
d.write_csv(input)